**TOPSIS Analysis of GPU Compute Instances for HPC and AI in the Cloud**

This Jupyter notebook contains the implementation of the Technique for Order of Preference by Similarity to Ideal Solution (TOPSIS), a Multi Criteria Decision Making (MCDM) method for evaluating and ranking GPU compute instances for HPC and AI from various cloud providers. The notebook guides you through the process of data preparation, criteria weighting, and application of the TOPSIS algorithm. Additionally, it includes sensitivity analysis to explore the impact of varying criteria weights, bootstrap analysis to assess the stability of the rankings, and non-parametric tests to evaluate the consistency of the results. The notebook is designed to be a comprehensive tool for researchers and practitioners looking to make informed decisions about GPU compute instance selection for HPC and AI in cloud computing environments.

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata, friedmanchisquare
import seaborn as sns

# Set display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# Define the TOPSIS function with epsilon to avoid division by zero
def topsis(raw_data, weights, benefit_categories, epsilon=1e-10):
    m, n = raw_data.shape
    # Normalize the raw data
    divisors = np.sqrt(np.sum(raw_data ** 2, axis=0))
    normalized_data = raw_data / divisors

    # Apply weights
    weighted_data = normalized_data * weights

    # Determine Ideal and Negative Ideal Solutions
    ideal_solution = np.zeros(n)
    negative_ideal_solution = np.zeros(n)
    for j in range(n):
        if j in benefit_categories:
            ideal_solution[j] = np.max(weighted_data[:, j])
            negative_ideal_solution[j] = np.min(weighted_data[:, j])
        else:
            ideal_solution[j] = np.min(weighted_data[:, j])
            negative_ideal_solution[j] = np.max(weighted_data[:, j])

    # Calculate distances
    dist_to_ideal = np.sqrt(np.sum((weighted_data - ideal_solution) ** 2, axis=1))
    dist_to_negative_ideal = np.sqrt(np.sum((weighted_data - negative_ideal_solution) ** 2, axis=1))

    # Calculate TOPSIS scores with epsilon to prevent division by zero
    scores = dist_to_negative_ideal / (dist_to_ideal + dist_to_negative_ideal + epsilon)
    return scores

**Identification of Criteria and Weights**

The initial phase of the TOPSIS methodology involves determining and defining the criteria.

**Requirements:**

For this analysis, the following information is consistently provided:

The scores for each alternative across various categories.

*   The scores for each alternative across various categories.
*   The importance or weights assigned to each category.

Note: Categories may be classified as either beneficial, where maximizing their contribution is desired, or as cost categories, where minimizing their impact is preferred.

In [ ]:
# Identification of Criteria and Weights
categories = np.array(["Number of GPU (G)", "Number of Physical CPU Cores (C)", "GPU Memory (GM)", "CPU Memory (CM)", "GPU FP64 Performance (GP)", "On-Demand Hourly Cost (HC)"])
alternatives = np.array(["AWS p5.48xlarge", "AWS p4de.24xlarge", "AWS p4d.24xlarge", "GCP a3-highgpu-8g", "GCP a2-ultragpu-8g", "GCP a2-ultragpu-4g", "GCP a2-ultragpu-2g", "GCP a2-ultragpu-1g", "GCP a2-megagpu-16g", "GCP a2-highgpu-8g", "GCP a2-highgpu-4g", "GCP a2-highgpu-2g", "GCP a2-highgpu-1g", "Azure ND96isr_H100_v5", "Azure NC80adis_H100_v5", "Azure NC40ads_H100_v5", "Azure ND96amsr_A100_v4", "Azure ND96asr_A100_v4", "Azure NC96ads_A100_v4", "Azure NC48ads_A100_v4", "Azure NC24ads_A100_v4", "OCI BM.GPU.H100.8", "OCI BM.GPU.A100-v2.8", "OCI BM.GPU4.8"])
raw_data = np.array([
    [8, 96, 640, 2048, 272, 98.32],
    [8, 48, 640, 1152, 77.6, 53.09472],
    [8, 48, 320, 1152, 77.6, 32.7726],
    [8, 104, 640, 1872, 272, 88.5057808],
    [8, 48, 640, 1360, 77.6, 40.5503836],
    [4, 24, 320, 680, 38.8, 20.2751918],
    [2, 12, 160, 340, 19.4, 10.137589],
    [1, 6, 80, 170, 9.7, 5.06879452],
    [16, 48, 640, 1360, 155.2, 55.7395068],
    [8, 48, 320, 680, 77.6, 29.3870822],
    [4, 24, 160, 340, 38.8, 14.6935342],
    [2, 12, 80, 170, 19.4, 7.34676712],
    [1, 6, 40, 85, 9.7, 3.67338356],
    [8, 96, 640, 1900, 272, 98.32],
    [2, 80, 188, 640, 60, 13.96],
    [1, 40, 94, 320, 30, 6.98],
    [8, 96, 640, 1900, 77.6, 32.77],
    [8, 96, 320, 900, 77.6, 27.197],
    [4, 96, 320, 880, 38.8, 14.692],
    [2, 48, 160, 440, 19.4, 7.346],
    [1, 24, 80, 220, 9.7, 3.673],
    [8, 112, 640, 2048, 272, 80],
    [8, 128, 640, 2048, 77.6, 32],
    [8, 64, 320, 2048, 77.6, 24.4],
])

initial_weights = np.array([0.15, 0.10, 0.15, 0.10, 0.25, 0.25])
benefit_categories = set([0, 1, 2, 3, 4])

# Display raw data and weights
raw_data_df = pd.DataFrame(data=raw_data, index=alternatives, columns=categories)
weights_df = pd.DataFrame(data=initial_weights, index=categories, columns=["Weights"])

print("Raw Data:")
display(raw_data_df)
print("Initial Weights:")
display(weights_df)

Raw Data:


,Number of GPU (G),Number of Physical CPU Cores (C),GPU Memory (GM),CPU Memory (CM),GPU FP64 Performance (GP),On-Demand Hourly Cost (HC)
AWS p5.48xlarge,8.0,96.0,640.0,2048.0,272.0,98.320000
AWS p4de.24xlarge,8.0,48.0,640.0,1152.0,77.6,53.094720
AWS p4d.24xlarge,8.0,48.0,320.0,1152.0,77.6,32.772600
GCP a3-highgpu-8g,8.0,104.0,640.0,1872.0,272.0,88.505781
GCP a2-ultragpu-8g,8.0,48.0,640.0,1360.0,77.6,40.550384
GCP a2-ultragpu-4g,4.0,24.0,320.0,680.0,38.8,20.275192
GCP a2-ultragpu-2g,2.0,12.0,160.0,340.0,19.4,10.137589
GCP a2-ultragpu-1g,1.0,6.0,80.0,170.0,9.7,5.068795
GCP a2-megagpu-16g,16.0,48.0,640.0,1360.0,155.2,55.739507
GCP a2-highgpu-8g,8.0,48.0,320.0,680.0,77.6,29.387082


Initial Weights:


,Weights
Number of GPU (G),0.15
Number of Physical CPU Cores (C),0.10
GPU Memory (GM),0.15
CPU Memory (CM),0.10
GPU FP64 Performance (GP),0.25
On-Demand Hourly Cost (HC),0.25


**Normalization of Data**

Normalization is essential to bring all criteria to a common scale, ensuring that each criterion contributes proportionally to the decision-making process. This step involves transforming the raw data for each criterion into a dimensionless value between 0 and 1. Various normalization techniques, such as min-max normalization or z-score normalization, can be applied depending on the nature of the data.


In [ ]:
# Normalize the raw data
m, n = raw_data.shape
divisors = np.empty(n)
for j in range(n):
    column = raw_data[:, j]
    divisors[j] = np.sqrt(column @ column)
normalized_data = raw_data / divisors

# Normalize the weights to ensure that they sum up to 1
weights = initial_weights / np.sum(initial_weights)

normalized_data_df = pd.DataFrame(data=normalized_data, index=alternatives, columns=categories)

print("Normalized Data:")
display(normalized_data_df)

Normalized Data:


,Number of GPU (G),Number of Physical CPU Cores (C),GPU Memory (GM),CPU Memory (CM),GPU FP64 Performance (GP),On-Demand Hourly Cost (HC)
AWS p5.48xlarge,0.242091,0.284537,0.303642,0.334819,0.442043,0.449435
AWS p4de.24xlarge,0.242091,0.142269,0.303642,0.188336,0.126112,0.242703
AWS p4d.24xlarge,0.242091,0.142269,0.151821,0.188336,0.126112,0.149808
GCP a3-highgpu-8g,0.242091,0.308249,0.303642,0.306046,0.442043,0.404572
GCP a2-ultragpu-8g,0.242091,0.142269,0.303642,0.222341,0.126112,0.185362
GCP a2-ultragpu-4g,0.121046,0.071134,0.151821,0.111170,0.063056,0.092681
GCP a2-ultragpu-2g,0.060523,0.035567,0.075911,0.055585,0.031528,0.046340
GCP a2-ultragpu-1g,0.030261,0.017784,0.037955,0.027793,0.015764,0.023170
GCP a2-megagpu-16g,0.484182,0.142269,0.303642,0.222341,0.252224,0.254793
GCP a2-highgpu-8g,0.242091,0.142269,0.151821,0.111170,0.126112,0.134333


The weights are normalized to ensure that they sum up to 1.

In [ ]:
# Weighted normalized decision matrix
weighted_data = normalized_data * weights

weighted_data_df = pd.DataFrame(data=weighted_data, index=alternatives, columns=categories)

print("Weighted Normalized Data:")
display(weighted_data_df)

Weighted Normalized Data:


,Number of GPU (G),Number of Physical CPU Cores (C),GPU Memory (GM),CPU Memory (CM),GPU FP64 Performance (GP),On-Demand Hourly Cost (HC)
AWS p5.48xlarge,0.036314,0.028454,0.045546,0.033482,0.110511,0.112359
AWS p4de.24xlarge,0.036314,0.014227,0.045546,0.018834,0.031528,0.060676
AWS p4d.24xlarge,0.036314,0.014227,0.022773,0.018834,0.031528,0.037452
GCP a3-highgpu-8g,0.036314,0.030825,0.045546,0.030605,0.110511,0.101143
GCP a2-ultragpu-8g,0.036314,0.014227,0.045546,0.022234,0.031528,0.046340
GCP a2-ultragpu-4g,0.018157,0.007113,0.022773,0.011117,0.015764,0.023170
GCP a2-ultragpu-2g,0.009078,0.003557,0.011387,0.005559,0.007882,0.011585
GCP a2-ultragpu-1g,0.004539,0.001778,0.005693,0.002779,0.003941,0.005793
GCP a2-megagpu-16g,0.072627,0.014227,0.045546,0.022234,0.063056,0.063698
GCP a2-highgpu-8g,0.036314,0.014227,0.022773,0.011117,0.031528,0.033583


**Determination of Ideal Solution and Negative Ideal Solution**

Ideal Solution and Negative Ideal Solution are key concepts used to evaluate alternatives based on their distance from these ideal points.

In [ ]:
# Determine the Ideal and Negative Ideal Solutions
a_pos = np.zeros(n)
a_neg = np.zeros(n)
for j in range(n):
    column = weighted_data[:, j]
    max_val = np.max(column)
    min_val = np.min(column)

    if j in benefit_categories:
        a_pos[j] = max_val
        a_neg[j] = min_val
    else:
        a_pos[j] = min_val
        a_neg[j] = max_val

ideal_df = pd.DataFrame(data=[a_pos, a_neg], index=["Ideal Solution", "Negative Ideal Solution"], columns=categories)
print("Ideal and Negative Ideal Solutions:")
display(ideal_df)

Ideal and Negative Ideal Solutions:


,Number of GPU (G),Number of Physical CPU Cores (C),GPU Memory (GM),CPU Memory (CM),GPU FP64 Performance (GP),On-Demand Hourly Cost (HC)
Ideal Solution,0.072627,0.037938,0.045546,0.033482,0.110511,0.004197
Negative Ideal Solution,0.004539,0.001778,0.002847,0.001390,0.003941,0.112359


**Calculation of Similarity Scores**

The core of TOPSIS lies in the calculation of similarity scores for each alternative with respect to the ideal and negative ideal solutions. The ideal solution represents the maximum (or minimum, depending on the nature of the criterion) values for each criterion, while the negative ideal solution represents the minimum (or maximum) values.

In [ ]:
# Calculate the similarity scores
sp = np.zeros(m)
sn = np.zeros(m)
cs = np.zeros(m)

for i in range(m):
    diff_pos = weighted_data[i] - a_pos
    diff_neg = weighted_data[i] - a_neg
    sp[i] = np.sqrt(diff_pos @ diff_pos)
    sn[i] = np.sqrt(diff_neg @ diff_neg)
    cs[i] = sn[i] / (sp[i] + sn[i])

similarity_scores_df = pd.DataFrame(data=zip(sp, sn), index=alternatives, columns=["S+", "S-"])
print("Similarity Scores:")
display(similarity_scores_df)

Similarity Scores:


,S+,S-
AWS p5.48xlarge,0.114488,0.126220
AWS p4de.24xlarge,0.107348,0.082002
AWS p4d.24xlarge,0.099791,0.090763
GCP a3-highgpu-8g,0.103808,0.126542
GCP a2-ultragpu-8g,0.100109,0.092422
GCP a2-ultragpu-4g,0.119469,0.093808
GCP a2-ultragpu-2g,0.133246,0.101415
GCP a2-ultragpu-1g,0.140834,0.106613
GCP a2-megagpu-16g,0.080505,0.113627
GCP a2-highgpu-8g,0.100007,0.092860


**Ranking of Alternatives**

The final step involves ranking the alternatives based on their relative closeness to the ideal solution and distance from the anti-ideal solution.

In [ ]:
# Ranking of alternatives
initial_ranks = rankdata(-cs)
ranking_df = pd.DataFrame(data=zip(cs, initial_ranks), index=alternatives, columns=["TOPSIS Score", "Initial Rank"]).sort_values(by="Initial Rank")
print("Initial Ranking of Alternatives (Descending Order):")
display(ranking_df)

Initial Ranking of Alternatives (Descending Order):


,TOPSIS Score,Initial Rank
GCP a2-megagpu-16g,0.585309,1.0
OCI BM.GPU.H100.8,0.576951,2.0
GCP a3-highgpu-8g,0.549348,3.0
OCI BM.GPU.A100-v2.8,0.538071,4.0
Azure ND96amsr_A100_v4,0.526137,5.0
AWS p5.48xlarge,0.524370,6.0
Azure ND96isr_H100_v5,0.523139,7.0
OCI BM.GPU4.8,0.520834,8.0
Azure ND96asr_A100_v4,0.505589,9.0
GCP a2-highgpu-8g,0.481469,10.0


**Sensitivity Analysis**

Sensitivity analysis in the context of TOPSIS is performed to evaluate the robustness of the rankings by examining how variations in the criteria weights affect the results. This analysis ensures that the final rankings are reliable and not overly sensitive to changes in the assigned weights.

In [ ]:
# Sensitivity Analysis: Varying weights for each criterion
def sensitivity_analysis(raw_data, initial_weights, benefit_categories, alternatives):
    sensitivities = {}
    # Obtain initial ranking with current weights
    base_scores = topsis(raw_data, initial_weights, benefit_categories)
    base_ranking = rankdata(-base_scores)

    for i in range(len(initial_weights)):
        altered_weights = initial_weights.copy()
        for delta in np.linspace(-0.1, 0.1, 5):  # vary weights by ±10%
            if 0 <= initial_weights[i] + delta <= 1:
                altered_weights[i] = initial_weights[i] + delta
                # Ensure the weights sum to 1
                altered_weights /= np.sum(altered_weights)
                scores = topsis(raw_data, altered_weights, benefit_categories)
                ranking = rankdata(-scores)
                # Store the result using base_ranking as reference
                sensitivity_key = (i, delta)
                sensitivities[sensitivity_key] = pd.Series(ranking, index=alternatives)

    # Convert sensitivity results to DataFrame and align columns with initial ranking
    sensitivity_df = pd.DataFrame(sensitivities).T
    sensitivity_df.columns = alternatives  # Ensure correct column names for alternatives
    sensitivity_df.index.names = ['Criterion', 'Delta']
    sensitivity_df = sensitivity_df[ranking_df.sort_values("Initial Rank").index]

    return sensitivity_df

# Perform sensitivity analysis
sensitivity_df = sensitivity_analysis(raw_data, initial_weights, benefit_categories, alternatives)

print("Sensitivity Analysis:")
display(sensitivity_df)

Sensitivity Analysis:


GCP a2-megagpu-16g  OCI BM.GPU.H100.8  GCP a3-highgpu-8g  \
Criterion Delta                                                             
0         -0.10                 4.0                1.0                2.0   
          -0.05                 3.0                1.0                2.0   
           0.00                 1.0                2.0                3.0   
           0.05                 1.0                2.0                3.0   
           0.10                 1.0                2.0                3.0   
1         -0.10                 1.0                2.0                3.0   
          -0.05                 1.0                2.0                3.0   
           0.00                 1.0                2.0                3.0   
           0.05                 2.0                1.0                3.0   
           0.10                 4.0                1.0                3.0   
2         -0.10                 1.0                2.0                3.0   
          -0.05                 1.0                2.0                3.0   
           0.00                 1.0                2.0                3.0   
           0.05                 1.0                2.0                3.0   
           0.10                 1.0                2.0                3.0   
3         -0.10                 1.0                2.0                3.0   
          -0.05                 1.0                2.0                3.0   
           0.00                 1.0                2.0                3.0   
           0.05                 1.0                2.0                3.0   
           0.10                 2.0                1.0                4.0   
4         -0.10                 3.0               12.0               22.0   
          -0.05                 1.0                6.0               12.0   
           0.00                 1.0                2.0                5.0   
           0.05                 2.0                1.0                3.0   
           0.10                 5.0                1.0                2.0   
5         -0.10                 5.0                1.0                2.0   
          -0.05                 3.0                1.0                2.0   
           0.00                 1.0                2.0                3.0   
           0.05                 1.0                5.0                7.0   
           0.10                 7.0               20.0               21.0   

                 OCI BM.GPU.A100-v2.8  Azure ND96amsr_A100_v4  \
Criterion Delta                                                 
0         -0.10                   3.0                     5.0   
          -0.05                   4.0                     5.0   
           0.00                   4.0                     5.0   
           0.05                   4.0                     5.0   
           0.10                   4.0                     7.0   
1         -0.10                   4.0                     7.0   
          -0.05                   4.0                     7.0   
           0.00                   4.0                     5.0   
           0.05                   4.0                     5.0   
           0.10                   2.0                     5.0   
2         -0.10                   5.0                     9.0   
          -0.05                   4.0                     8.0   
           0.00                   4.0                     5.0   
           0.05                   4.0                     5.0   
           0.10                   4.0                     5.0   
3         -0.10                   4.0                     7.0   
          -0.05                   4.0                     5.0   
           0.00                   4.0                     5.0   
           0.05                   4.0                     5.0   
           0.10                   3.0                     5.0   
4         -0.10                   1.0                     2.0   
          -0.05                   2.0                    

**Bootstrapping Analysis**

Bootstrapping analysis is employed to evaluate the variability and stability of TOPSIS rankings by generating multiple resamples of the decision matrix and recalculating the TOPSIS scores for each resample. This approach helps in understanding the distribution of rankings and assessing the robustness of the decision outcomes.

In [ ]:
# Bootstrapping Analysis: Generating bootstrap samples and calculating TOPSIS scores
def bootstrap_analysis(raw_data, initial_weights, benefit_categories, num_samples=1000):
    m, n = raw_data.shape
    bootstrap_scores = np.zeros((num_samples, m))

    for i in range(num_samples):
        bootstrap_sample_indices = np.random.choice(m, m, replace=True)
        bootstrap_sample = raw_data[bootstrap_sample_indices]
        bootstrap_scores[i] = topsis(bootstrap_sample, initial_weights, benefit_categories)

    return bootstrap_scores

bootstrap_scores = bootstrap_analysis(raw_data, initial_weights, benefit_categories)

# Analyzing the bootstrap results
bootstrap_ranks = np.array([rankdata(-scores) for scores in bootstrap_scores])
bootstrap_mean_ranks = np.mean(bootstrap_ranks, axis=0)
bootstrap_rank_intervals = np.percentile(bootstrap_ranks, [2.5, 97.5], axis=0)

# Display bootstrap analysis results
bootstrap_df = pd.DataFrame({
    "TOPSIS Score": topsis(raw_data, initial_weights, benefit_categories),
    "Initial Rank": initial_ranks,
    "Mean Rank": bootstrap_mean_ranks,
    "2.5% Rank": bootstrap_rank_intervals[0],
    "97.5% Rank": bootstrap_rank_intervals[1]
}, index=alternatives).sort_values(by="Initial Rank")

print("Bootstrap Analysis Results (Descending Order):")
display(bootstrap_df)

Bootstrap Analysis Results (Descending Order):


,TOPSIS Score,Initial Rank,Mean Rank,2.5% Rank,97.5% Rank
GCP a2-megagpu-16g,0.585309,1.0,12.3940,1.0000,24.0000
OCI BM.GPU.H100.8,0.576951,2.0,12.4755,1.0000,23.5125
GCP a3-highgpu-8g,0.549348,3.0,12.7045,1.0000,24.0000
OCI BM.GPU.A100-v2.8,0.538071,4.0,12.2720,1.5000,23.5000
Azure ND96amsr_A100_v4,0.526137,5.0,12.6255,1.5000,24.0000
AWS p5.48xlarge,0.524370,6.0,12.6230,1.5000,23.5000
Azure ND96isr_H100_v5,0.523139,7.0,12.2835,1.0000,23.5000
OCI BM.GPU4.8,0.520834,8.0,12.5640,1.0000,23.5000
Azure ND96asr_A100_v4,0.505589,9.0,12.6260,1.5000,23.5000
GCP a2-highgpu-8g,0.481469,10.0,12.3425,1.0000,24.0000


**Non-Parametric Tests**

Non-parametric tests are utilized to evaluate the statistical significance of the differences in rankings obtained from the bootstrapping analysis. These tests do not assume a specific distribution for the data and are particularly useful for analyzing ordinal rankings.

In [ ]:
# Non-parametric Tests: Friedman Test
def friedman_test(bootstrap_ranks):
    # Perform the Friedman test
    stat, p = friedmanchisquare(*bootstrap_ranks.T)
    return stat, p

# Perform the Friedman test
stat, p = friedman_test(bootstrap_ranks)
print(f"Friedman Test Statistic: {stat}, p-value: {p}")

# Adding Friedman Test p-value to summary table
bootstrap_df["Friedman Test p-value"] = p
print("Final Summary Table with Friedman Test p-value (Descending Order):")
display(bootstrap_df)

Friedman Test Statistic: 11.90709955410787, p-value: 0.9718789771808888
Final Summary Table with Friedman Test p-value (Descending Order):


,TOPSIS Score,Initial Rank,Mean Rank,2.5% Rank,97.5% Rank,Friedman Test p-value
GCP a2-megagpu-16g,0.585309,1.0,12.3940,1.0000,24.0000,0.971879
OCI BM.GPU.H100.8,0.576951,2.0,12.4755,1.0000,23.5125,0.971879
GCP a3-highgpu-8g,0.549348,3.0,12.7045,1.0000,24.0000,0.971879
OCI BM.GPU.A100-v2.8,0.538071,4.0,12.2720,1.5000,23.5000,0.971879
Azure ND96amsr_A100_v4,0.526137,5.0,12.6255,1.5000,24.0000,0.971879
AWS p5.48xlarge,0.524370,6.0,12.6230,1.5000,23.5000,0.971879
Azure ND96isr_H100_v5,0.523139,7.0,12.2835,1.0000,23.5000,0.971879
OCI BM.GPU4.8,0.520834,8.0,12.5640,1.0000,23.5000,0.971879
Azure ND96asr_A100_v4,0.505589,9.0,12.6260,1.5000,23.5000,0.971879
GCP a2-highgpu-8g,0.481469,10.0,12.3425,1.0000,24.0000,0.971879
